In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import sys
sys.path.insert(1, '../rtsvg')
from rtsvg import *
rt = RACETrack()

In [ ]:
_lu_ = {'dt':['2024-01-15 12:23:34', '2023-12-19 00:00:59', '2024-02-05 23:59:59', '2024-02-21 02:03:21']}
df    = pl.DataFrame(_lu_)
df_pd = pd.DataFrame(_lu_)
df    = rt.columnsAreTimestamps(df,    'dt')
df_pd = rt.columnsAreTimestamps(df_pd, 'dt')
df = df.sort('dt')
t_min, t_max = df_pd['dt'].min(), df_pd['dt'].max()
for i in range(len(rt.time_rezes)):
    if rt.time_rezes[i] == 's' or rt.time_rezes[i] == '15s': continue
    pd_time_rez = rt.time_rezes       [i]
    pl_time_rez = rt.time_rezes_polars[i]
    if   pd_time_rez == '100YS':
        century = round(t_min.year/100)*100
        t_min_fixed = pd.Timestamp(century, t_min.month, t_min.day, t_min.hour, t_min.minute, t_min.second, t_min.microsecond, t_min.tzinfo)
        tmp_df = pd.DataFrame({'dt':[t_min_fixed,t_max]})
    elif pd_time_rez == '10YS':
        decade = round(t_min.year/10)*10
        t_min_fixed = pd.Timestamp(decade, t_min.month, t_min.day, t_min.hour, t_min.minute, t_min.second, t_min.microsecond, t_min.tzinfo)
        tmp_df = pd.DataFrame({'dt':[t_min_fixed,t_max]})
    elif pd_time_rez == '1W-MON':
        if t_min.weekday() == 0: t_min_fixed = t_min
        else:                    t_min_fixed = t_min - pd.Timedelta(7, 'D')
        tmp_df = pd.DataFrame({'dt':[t_min_fixed,t_max]})
    else:
        tmp_df = pd.DataFrame({'dt':[t_min,t_max]})
    df_groupby  = tmp_df.groupby(pd.Grouper(key='dt', freq=pd_time_rez))
    this_lu, first_was = {}, None
    for k, k_df in df_groupby: 
        if len(this_lu) == 0: first_was = k
        this_lu[k] = len(this_lu)
    all_lus_found = True
    for k, k_df in df.group_by_dynamic('dt', every=pl_time_rez):
        if k[0] not in this_lu:
            print(f'this was not found: {k[0]} [polars] .... first was: {first_was} [pandas]')
            all_lus_found = False
            break

    print(f'{pd_time_rez:10} | {len(this_lu)=:10} | {all_lus_found}')